## Código original

In [2]:
import random
import sys
import time

def calc_pi(N):
    M = 0
    for i in range(N):
    # Simulate impact coordinates
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
    # True if impact happens inside the circle
        if x**2 + y**2 < 1.0:
            M += 1
    return 4 * M / N


num_trials = 10**6
#num_trials = int(sys.argv[1])

t0 = time.time()
pi = calc_pi(num_trials)
t1 = time.time()

print("\n \t Computing pi in serial: \n")
print("\t For %d trials, pi = %f\n" % (num_trials,pi))

%timeit -r3 calc_pi(num_trials)
t_pi = t1 - t0


 	 Computing pi in serial: 

	 For 1000000 trials, pi = 3.142468

491 ms ± 1.16 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### Mejora con NumPy

In [3]:
import numpy as np
import time

def calc_pi_numpy(N):
    # Generamos vectores de puntos aleatorios
    x = np.random.uniform(-1, 1, N)
    y = np.random.uniform(-1, 1, N)

    inside = np.sum(x*x + y*y < 1.0)
    return 4 * inside / N

t0 = time.time()
pi_numpy = calc_pi_numpy(num_trials)
t1 = time.time()

print("Pi con NumPy =", pi_numpy)
print("Tiempo NumPy =", t1 - t0, "s")
t_numpy=t1-t0

Pi con NumPy = 3.142376
Tiempo NumPy = 0.028436660766601562 s


### Mejora también con Numba y paralelización

In [4]:
from numba import njit, prange
import numpy as np
import time

@njit(parallel=True)
def calc_pi_numba_parallel(N):
    inside = 0
    for i in prange(N):
        x = np.random.random() * 2 - 1
        y = np.random.random() * 2 - 1
        if x*x + y*y < 1.0:
            inside += 1
    return 4 * inside / N

# Compilación JIT
calc_pi_numba_parallel(1000)

t0 = time.time()
pi_par = calc_pi_numba_parallel(num_trials)
t1 = time.time()

print("Pi con Numba paralelo =", pi_par)
print("Tiempo Numba paralelo =", t1 - t0, "s")
t_par = t1 - t0


Pi con Numba paralelo = 3.14072
Tiempo Numba paralelo = 0.0014483928680419922 s


In [5]:
speed_numpy_vs_python = t_pi / t_numpy
speed_numba_par_vs_python = t_pi / t_par

print("=== FACTORES DE ACELERACIÓN ===")
print(f"NumPy vs Python puro: {speed_numpy_vs_python:.2f}× más rápido")
print(f"Numba paralelizado vs Python puro: {speed_numba_par_vs_python:.2f}× más rápido")

=== FACTORES DE ACELERACIÓN ===
NumPy vs Python puro: 17.21× más rápido
Numba paralelizado vs Python puro: 337.82× más rápido


### Resultados de la ejecución en hpc-bio-nikola-cpu

[alumno18@ibsen lab-python]$ cat slurm-pi-python-12767.out 
Ejecutando cálculo de pi con Numba en nikola

 	 Computing pi in serial: 

	 For 1000000 trials, pi = 3.144016

367 ms ± 587 μs per loop (mean ± std. dev. of 3 runs, 1 loop each)

Pi con NumPy = 3.137552

Tiempo NumPy = 0.021308422088623047 s

Pi con Numba paralelo = 3.142148

Tiempo Numba paralelo = 0.0016460418701171875 s

=== FACTORES DE ACELERACIÓN ===

NumPy vs Python puro: 17.24× más rápido

Numba paralelizado vs Python puro: 223.12× más rápido